In [1]:
# Import necessary libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

import tensorflow as tf

In [5]:
# Load the data

base_file_path = 'C:/Users/nikoLocal/Documents/Opencampus/Machine_Vision_challenge_data/'
image_path = base_file_path + '/input_train/input_train'

label_df = pd.read_csv(os.path.join(base_file_path, 'Y_train_eVW9jym.csv'))

label_df.head()

,Unnamed: 0,filename,window,lib,Label
0,0,15b3bab7c186fd35b65df777890c427dd243feacbb85dd...,2003,Die01,Missing
1,1,1856617e1ac2d821a46a41b938818f0169342226a78f93...,2003,Die01,GOOD
2,2,19066cce773b3a092ebf4311b11858aa653da6f8274957...,2003,Die01,Missing
3,3,19c10caf4b24284e1748caed62d94cbb689d6b379b1cf5...,2003,Die01,GOOD
4,4,1a627426d55a668df8bcd381a7fa87b620481995b6755f...,2003,Die01,Missing


In [8]:
label_df.duplicated(subset='filename').value_counts()
# no duplicated file names

False    8278
Name: count, dtype: int64

In [10]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

rescale = 1.0/255.0

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    shear_range=16,
    rescale=rescale,
    validation_split=0.25)

target_size = (200,200)
class_mode = 'categorical'
batch_size = 64

train_generator = train_datagen.flow_from_dataframe(
    label_df,
    image_path,
    x_col='filename',
    y_col='Label',
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='training')

#why are only 6209 file names found? - because of subset!

Found 6209 validated image filenames belonging to 6 classes.


In [29]:
#subset

total_item_num = 1200 #rough number of GOOD images

GOOD_df = label_df[label_df['Label'] == 'GOOD']
Missing_df = label_df[label_df['Label'] == 'Missing']

GOOD_df_sampled = GOOD_df.sample(n=total_item_num)
Missing_df_sampled = Missing_df.sample(n=total_item_num)

BalancedDF = pd.concat([GOOD_df_sampled,Missing_df_sampled],axis=0)
#randomize order
BalancedDF = BalancedDF.sort_values('filename') # filenames have no particular order
#BalancedDF.head()

train_generator2 = train_datagen.flow_from_dataframe(
    BalancedDF,
    image_path,
    x_col='filename',
    y_col='Label',
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset=None)

Found 2400 validated image filenames belonging to 2 classes.


In [20]:
'''
from sklearn.utils import class_weight as cw

def get_weight(y):
    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current

a = get_weight(train_generator.classes)
'''

TypeError: too many positional arguments

TypeError: too many positional arguments